In [1]:
library(tidyverse)

-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.3.2 --
v ggplot2 3.3.6      v purrr   0.3.4 
v tibble  3.1.6      v dplyr   1.0.10
v tidyr   1.2.0      v stringr 1.4.1 
v readr   2.1.2      v forcats 0.5.1 
-- Conflicts ---------------------------------------------------------------------------------- tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()


In [2]:
sales_Data <- read.csv('competitive-data-science-predict-future-sales/sales_train.csv')
testData <- read.csv('competitive-data-science-predict-future-sales/test.csv')
itemsData <- read.csv('competitive-data-science-predict-future-sales/items.csv')
shopsData <- read.csv('competitive-data-science-predict-future-sales/shops.csv')
itemsCatData <- read.csv('competitive-data-science-predict-future-sales/item_categories.csv')

In [3]:
sales_Data

date,date_block_num,shop_id,item_id,item_price,item_cnt_day
<chr>,<int>,<int>,<int>,<dbl>,<dbl>
02.01.2013,0,59,22154,999.00,1
03.01.2013,0,25,2552,899.00,1
05.01.2013,0,25,2552,899.00,-1
06.01.2013,0,25,2554,1709.05,1
15.01.2013,0,25,2555,1099.00,1
10.01.2013,0,25,2564,349.00,1
02.01.2013,0,25,2565,549.00,1
04.01.2013,0,25,2572,239.00,1
11.01.2013,0,25,2572,299.00,1
